In [1]:
from src.llm.generator import LLamaCppGeneratorComponent

/Users/esp.py/Projects/Personal/congo-news-summarizer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = "You are a french news reporter"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [13]:
%load_ext dotenv

In [14]:
%dotenv ./.env_local -o

In [15]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [16]:
llama_cpp_generator._ping_api()

True

In [17]:
summarization_prompt = """
Give a short summary 2 to 3 sentences in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

### Download the File

In [18]:
from src.shared.cloud_storage import BackBlazeCloudStorage

In [19]:
cloud_storage = BackBlazeCloudStorage(environment="local")

In [20]:
filename = "news-clusters-2024-11-20-to-2024-11-18.csv"

In [21]:
BUCKET_NAME = "congonews-clusters"

In [22]:
import pandas as pd

In [23]:
from tempfile import NamedTemporaryFile

In [28]:
today_news_file = cloud_storage.download_by_name(
    bucket_name=BUCKET_NAME, file_name=filename)
with NamedTemporaryFile(delete=True, suffix=".csv") as temp_file:
    today_news_file.save_to(temp_file.name)
    today_news_df = pd.read_csv(temp_file.name, sep="|", index_col=0)

In [29]:
today_news_df.head()

,database_id,content,title,posted_at,url,labels
0,31617542,Même s’il est prématuré de sonner le glas de l...,Révision ou changement de la Constitution : l...,2024-11-20 00:00:00,https://econewsrdc.com/revision-ou-changement-...,76
1,31262380,Les Léopards reçoivent les Walya de l’Ethiopie...,Elim/CAN 2025 : « nous sommes entrain d’ évalu...,2024-11-19 10:08:00,https://actualite.cd/2024/11/19/elimcan-2025-n...,29
3,31615265,"Dès lors, des élections législatives anticipée...",Bravo Sénégal !,2024-11-20 00:00:00,https://econewsrdc.com/bravo-senegal/,75
4,31615377,"Quand il veut être clair et direct, il passe p...",Steve Mbikayi titille Modeste Bahati,2024-11-20 00:00:00,https://econewsrdc.com/steve-mbikayi-titille-m...,76
5,31615389,"Le procès des «forages», désormais jugé par la...",Procès des «Forages» : tout chemin mène à l’an...,2024-11-20 00:00:00,https://econewsrdc.com/proces-des-forages-tout...,89


In [30]:
today_news_df = today_news_df.sort_values(by="labels")

In [31]:
today_news_df.sort_values(by="labels")

,database_id,content,title,posted_at,url,labels
92,31562427,En sa qualité de coordonnateur du Conseil prés...,Sud global/Chine : François Muamba en appelle ...,2024-11-20 00:00:00,https://www.mediacongo.net/article-actualite-1...,4
138,31636465,Au cours de son intervention aux assises lors ...,Global South Think Tanks : François Muamba Tsh...,2024-11-20 11:09:00,https://actualite.cd/index.php/2024/11/20/glob...,4
140,31000142,La Première Ministre de la République Démocrat...,Conférence Africa Accelerating 2024 : Judith S...,2024-11-18 00:00:00,https://www.mediacongo.net/article-actualite-1...,6
111,31562749,Le Gouvernement attend d'abord du Canada d'êtr...,Judith Suminwa appelle le Canada à soutenir le...,2024-11-20 00:00:00,https://www.mediacongo.net/article-actualite-1...,6
17,31345599,"novembre 18, 2024 RDC : “Je suis là pour obten...",RDC : “Je suis là pour obtenir des résultats” ...,2024-11-18 00:00:00,https://depeche.cd/2024/11/18/rdc-je-suis-la-p...,6
...,...,...,...,...,...,...
203,31288342,Après le moyen exceptionnel soulevé par la déf...,Instruction du procès « forages » : Mike Kasen...,2024-11-19 06:32:16,https://scooprdc.net/2024/11/19/instruction-du...,89
133,31261209,L’instruction du dossier des constructions des...,"Selon François Rubota, l’idée de construire 1 ...",2024-11-19 09:59:00,https://actualite.cd/2024/11/19/selon-francois...,89
32,30996093,L’affaire mettant en cause François Rubota et ...,Dossier forages : les avocats de François Rubo...,2024-11-18 12:56:00,https://actualite.cd/2024/11/18/dossier-forage...,89
82,31241267,La Cour de cassation siégeant en matière répre...,Dossier Forages: la Cour a pris en délibéré la...,2024-11-19 07:59:00,https://actualite.cd/index.php/2024/11/19/doss...,89


### Prompt

In [33]:
from pprint import pprint

In [34]:
summaries_16bits = []
for id, news_group in today_news_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(
        template_values={"content": news}, prompt_template=summarization_prompt)
    summaries_16bits.append(summary)
    pprint(summary)
    with pd.option_context('display.max_colwidth', None):
        display(news_group.title)
    print("---" * 10)

("Le 14 novembre dernier, lors du lancement officiel de l'Alliance des Groupes "
 'de réflexion du Sud Global à Nianjing en Chine, François Muamba Tshishimbi, '
 'coordonnateur du Conseil présidentiel de veille stratégique (CPVS), a '
 "rappelé la responsabilité historique de la Chine d'accompagner les pays en "
 'voie de développement dans leur vocation de diversifier leur base de '
 'production au profit des industries manufacturières et de services, plutôt '
 'que des exportations primaires traditionnelles. Il a invité la Chine à '
 'éviter que ces pays du Sud restent cantonner dans leur rôle classique de '
 'fournisseur des matières premières, ce qui renforcerait leur position '
 'défavorable dans la division internationale du travail. Pour François Muamba '
 'Tshishimbi, le changement de paradigme est la condition sine qua non pour '
 "développer et pérenniser la coopération inter Sud Global sur fond d'égalité, "
 'amitié sincère et davantage gagnant-gagnant. Il a également salué 

92                                                                                                  Sud global/Chine : François Muamba en appelle au renforcement de l'unité et la confiance mutuelle
138    Global South Think Tanks : François Muamba Tshishimbi appelle la chine à accompagner les pays en développement dans la diversification de produits d’industries manufacturières | Actualite.cd
Name: title, dtype: object

------------------------------
('La Première Ministre de la République Démocratique du Congo (RDC), Judith '
 'Suminwa Tuluka, est arrivée à Ottawa pour participer à la conférence Africa '
 'Accelerating 2024. Organisée par la Chambre de commerce Canada-Afrique et le '
 'gouvernement canadien, cette rencontre vise à renforcer les relations '
 'économiques entre la RDC et le Canada. La conférence, axée sur la création '
 "de partenariats stratégiques et durables, a pour objectif d'ouvrir de "
 "nouvelles opportunités d'investissement et de promouvoir une coopération "
 'bilatérale fondée sur des principes de bénéfices mutuels. En sa qualité de '
 'représentante de la RDC, la Première Ministre Suminwa Tuluka aura '
 "l'opportunité d'aborder plusieurs enjeux cruciaux, tels que l'amélioration "
 "de l'environnement des affaires en RDC, l'attraction d'investissements "
 "étrangers et le soutien aux secteurs stratégiques comme l'infrastructure, "
 "l'énergie et les ressources naturelles. Le 

140    Conférence Africa Accelerating 2024 : Judith Suminwa à Ottawa pour représenter la RDC et renforcer les partenariats gagnant-gagnant avec le Canada
111                                                                 Judith Suminwa appelle le Canada à soutenir le retour de la paix dans l'Est de la RDC
17                                                       RDC : “Je suis là pour obtenir des résultats” Judith Suminwa à la diaspora congolaise du Canada)
43                               RDC-Canada : Judith Suminwa présente les priorités de son Gouvernement au Président de la chambre des communes du Canada
Name: title, dtype: object

------------------------------
('La République Démocratique du Congo et le Fonds Monétaire International ont '
 'signé un accord préliminaire pour deux nouveaux programmes économiques et '
 'financiers. Le premier vise à favoriser la croissance et la diversification '
 'économique, tandis que le second soutient la lutte contre le changement '
 'climatique. Ces accords, soutenus par des facilités du FMI, sont prévus pour '
 "être approuvés par le Conseil d'Administration en janvier 2025. Le "
 "gouvernement a salué ces accords, soulignant l'importance de la mise en "
 'œuvre des réformes liées à ces programmes avec le Fonds Monétaire '
 'International.')


...

------------------------------
('Le ministre de la Justice et Garde des Sceaux, Constant Mutamba Tungunga, a '
 "ordonné au Procureur général près la Cour d'appel de Mbuji-Mayi de procéder "
 'à une enquête judiciaire contre les membres de la commission chargée de '
 "l'indemnisation des habitants des avenues Dodoma et Zaire, situées dans le "
 "quartier de la Plaine, autour de l'aéroport de Bipemba. Cette décision est "
 "motivée par la non-reception d'une somme de 3 millions de dollars américains "
 'alloués par la Banque Mondiale pour indemniser les propriétaires des maisons '
 "démolies suite aux travaux de modernisation de l'aéroport de Mbuji-Mayi.")


34     Kasaï-Oriental : Constant Mutamba ordonne l'instruction judiciaire contre la commission chargée d'indemniser les victimes de démolition autour de l'aéroport de Mbuji-Mayi | Actualite.cd
168                                     Kasaï-Oriental : Constant Mutamba ordonne des poursuites contre la commission chargée d'indemniser les victimes des démolitions des maisons à l’aéroport
Name: title, dtype: object

------------------------------
('### Procès des "forages" : Nicolas Kazadi à l\'honneur\n'
 '\n'
 'Le procès des "forages", désormais jugé par la Cour de cassation, prend un '
 "tournant crucial alors que les révélations autour du rôle de l'ancien "
 'ministre des Finances, Nicolas Kazadi, continuent de dominer les débats. '
 'Bien que le procureur général près la Cour de cassation, Firmin Mvonde, ait '
 'innocenté Nicolas Kazadi de toute implication, les accusations se font de '
 "plus en plus pressantes, notamment de la part de l'ex-ministre du "
 'Développement rural, François Rubota, qui réclame la comparution de '
 "l'ex-argentier national pour faire toute la lumière sur cette affaire.\n"
 '\n'
 'Le procès, qui porte sur un contrat de mise en place de stations de '
 'traitement d’eau, fait face à de nombreuses interrogations, notamment sur la '
 'modification du nombre de stations initialement prévues. Rubota a affirmé '
 "qu'il n’était en rien responsable de l'extension du projet

133                   Selon François Rubota, l’idée de construire 1 340 forages au lieu de 1000 prévus au contrat est venue de Nicolas Kazadi | Actualite.cd
32                                          Dossier forages : les avocats de François Rubota soulèvent l’incompétence de la Cour de cassation | Actualite.cd
82     Dossier Forages: la Cour a pris en délibéré la demande de liberté provisoire de l'entrepreneur Mike Kasenga, arrêt attendu ce mercredi | Actualite.cd
203                                          Instruction du procès « forages » : Mike Kasenga imperturbable et tout droit dans ses bottes devant les juges !
5                                                                   Procès des «Forages» : tout chemin mène à l’ancien ministre des Finances, Nicolas Kazadi
Name: title, dtype: object

------------------------------


I will keep both model in the server and use them. I will be benchmackring them.

Chat template without huggingface.

In [43]:
text = " Sample text"

In [44]:
chat_input = llama_cpp_generator.generate_chat_input(template_values={"content": text}, prompt_template=summarization_prompt)
chat_tokens = llama_cpp_generator.tokenizer.apply_chat_template(
            chat_input, tokenize=False, add_generation_prompt=True
        )

In [49]:
chat_tokens

"<|im_start|>system\nYou are a french news reporter<|im_end|>\n<|im_start|>user\n\nGive a short summary 2 to 3 sentences in french of the following document:\n Sample text\n Describes it in a style of a french new paper reporters.\n\nDon't summarize each document separately, the content in all the documents should be summarized.\n\nThe summary should be in french not in English<|im_end|>\n<|im_start|>assistant\n"

In [47]:
from jinja2 import Template

In [115]:
def generate_chat_prompt(messages, add_generation_prompt=False):
    """
    Generates a structured prompt based on a list of message dictionaries.

    Args:
    messages (list): A list of dictionaries, each with 'role' and 'content' keys.
    add_generation_prompt (bool): Whether to add a final assistant prompt block.

    Returns:
    str: The formatted prompt.
    """
    chat_template = "{%- if tools %}\n    {{- '<|im_start|>system\\n' }}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- messages[0]['content'] }}\n    {%- else %}\n        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}\n    {%- endif %}\n    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}\n    {%- for tool in tools %}\n        {{- \"\\n\" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}\n    {%- else %}\n        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}\n    {%- endif %}\n{%- endif %}\n{%- for message in messages %}\n    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) or (message.role == \"assistant\" and not message.tool_calls) %}\n        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}\n    {%- elif message.role == \"assistant\" %}\n        {{- '<|im_start|>' + message.role }}\n        {%- if message.content %}\n            {{- '\\n' + message.content }}\n        {%- endif %}\n        {%- for tool_call in message.tool_calls %}\n            {%- if tool_call.function is defined %}\n                {%- set tool_call = tool_call.function %}\n            {%- endif %}\n            {{- '\\n<tool_call>\\n{\"name\": \"' }}\n            {{- tool_call.name }}\n            {{- '\", \"arguments\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- '}\\n</tool_call>' }}\n        {%- endfor %}\n        {{- '<|im_end|>\\n' }}\n    {%- elif message.role == \"tool\" %}\n        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}\n            {{- '<|im_start|>user' }}\n        {%- endif %}\n        {{- '\\n<tool_response>\\n' }}\n        {{- message.content }}\n        {{- '\\n</tool_response>' }}\n        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}\n            {{- '<|im_end|>\\n' }}\n        {%- endif %}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\\n' }}\n{%- endif %}\n"
    template = Template(chat_template)
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)

In [116]:
chat_token_from_function = generate_chat_prompt(chat_input, add_generation_prompt=True)

In [117]:
chat_tokens

"<|im_start|>system\nYou are a french news reporter<|im_end|>\n<|im_start|>user\n\nGive a short summary 2 to 3 sentences in french of the following document:\n Sample text\n Describes it in a style of a french new paper reporters.\n\nDon't summarize each document separately, the content in all the documents should be summarized.\n\nThe summary should be in french not in English<|im_end|>\n<|im_start|>assistant\n"

In [118]:
chat_token_from_function


"<|im_start|>system\nYou are a french news reporter<|im_end|>\n<|im_start|>user\n\nGive a short summary 2 to 3 sentences in french of the following document:\n Sample text\n Describes it in a style of a french new paper reporters.\n\nDon't summarize each document separately, the content in all the documents should be summarized.\n\nThe summary should be in french not in English<|im_end|>\n<|im_start|>assistant\n"

In [119]:
assert chat_token_from_function == chat_tokens

In [ ]:
chat_template = """ {%- if tools %}\n    {{- '<|im_start|>system\\n' }}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- messages[0]['content'] }}\n    {%- else %}\n        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}\n    {%- endif %}\n    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}\n    {%- for tool in tools %}\n        {{- \"\\n\" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}\n    {%- else %}\n        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}\n    {%- endif %}\n{%- endif %}\n{%- for message in messages %}\n    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) or (message.role == \"assistant\" and not message.tool_calls) %}\n        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}\n    {%- elif message.role == \"assistant\" %}\n        {{- '<|im_start|>' + message.role }}\n        {%- if message.content %}\n            {{- '\\n' + message.content }}\n        {%- endif %}\n        {%- for tool_call in message.tool_calls %}\n            {%- if tool_call.function is defined %}\n                {%- set tool_call = tool_call.function %}\n            {%- endif %}\n            {{- '\\n<tool_call>\\n{\"name\": \"' }}\n            {{- tool_call.name }}\n            {{- '\", \"arguments\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- '}\\n</tool_call>' }}\n        {%- endfor %}\n        {{- '<|im_end|>\\n' }}\n    {%- elif message.role == \"tool\" %}\n        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}\n            {{- '<|im_start|>user' }}\n        {%- endif %}\n        {{- '\\n<tool_response>\\n' }}\n        {{- message.content }}\n        {{- '\\n</tool_response>' }}\n        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}\n            {{- '<|im_end|>\\n' }}\n        {%- endif %}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\\n' }}\n{%- endif %}\n,
"""